In [0]:
import pandas as pd

import numpy as np

import scipy as sp

import matplotlib.pyplot as plt

import sklearn as sk
from sklearn.model_selection import cross_val_score,KFold
#Leave-One-Out cross-validator(LOOCV)
from sklearn import model_selection,linear_model

from sklearn.metrics import mean_squared_error,mean_absolute_error

In [0]:
data=pd.read_csv('insurancee.csv')

In [0]:
data

In [0]:
data.isnull().sum()

In [0]:
data.info()

In [0]:
data.describe()

In [0]:
'''
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
data['sex']=le.fit_transform(data['sex'])
'''


In [0]:
'''
data['sex']
'''

In [0]:
'''
data['smoker']=le.fit_transform(data['smoker'])
'''

In [0]:
'''
data['smoker']
'''

In [0]:
'''
data.iloc[0:,0:]
'''

In [0]:
def map_smoking(column):
  mapped=[]
  for row in column:        
      if row=="yes":
          mapped.append(1)
      else:
          mapped.append(0)  
  return mapped
data["smoker_norm"]=map_smoking(data["smoker"])

In [0]:
data['smoker_norm']

In [0]:
nonnum_cols=[col for col in data.select_dtypes(include=["object"])]

In [0]:
nonnum_cols

In [0]:
def map_obese(column):
    mapped=[]
    for row in column:
        if row>30:
            mapped.append(1)
        else:
            mapped.append(0)
    return mapped
data["obese"]=map_obese(data["bmi"])

In [0]:
colnum=len(data.columns)-3
fig,ax=plt.subplots(colnum,1,figsize=(3,25))
ax[0].set_ylabel("charges")
p_vals={}
for ind,col in enumerate([i for i in data.columns if i not in ["smoker","region","charges","sex_norm"]]):  
    ax[ind].scatter(data[col],data.charges,s=5)
    ax[ind].set_xlabel(col)
    ax[ind].set_ylabel("charges")    
plt.show()

In [0]:
corr_vals=[]
collabel=[]
for col in [i for i in data.columns if i not in nonnum_cols]:
    
    p_val=sp.stats.pearsonr(data[col],data["charges"])
    corr_vals.append(np.abs(p_val[0]))
    print(col,": ",np.abs(p_val[0]))
    collabel.append(col)
plt.bar(range(1,len(corr_vals)+1),corr_vals)
plt.xticks(range(1,len(corr_vals)+1),collabel,rotation=45)
plt.ylabel("Absolute correlation")

In [0]:
cols_not_reg3=['age', 'obese', 'smoker_norm']

In [0]:
kf=KFold(n_splits=10, random_state=1, shuffle=True)
intercepts=[]
mses=[]
coefs=[]

for train_index, test_index in kf.split(data[cols_not_reg3]):
    
    lr=linear_model.LinearRegression()
    lr.fit(data[cols_not_reg3].iloc[train_index],data["charges"].iloc[train_index])
    lr_predictions=lr.predict(data[cols_not_reg3].iloc[test_index])
    
    lr_mse=mean_squared_error(data["charges"].iloc[test_index],lr_predictions)
    
    intercepts.append(lr.intercept_)
    
    coefs.append(lr.coef_)
    mses.append(lr_mse)

In [0]:
rmses=[x**.5 for x in mses]#root mean square error
avg_rmse=np.mean(rmses)
avg_intercept=np.mean(intercepts)
age_coefs=[]
obesity_coefs=[]
smoking_coefs=[]
for vals in coefs:
    #print vals[0]
    age_coefs.append(vals[0])
    obesity_coefs.append(vals[1])
    smoking_coefs.append(vals[2])
age_coef=np.mean(age_coefs)
obesity_coef=np.mean(obesity_coefs)
smoking_coef=np.mean(smoking_coefs)
print("a: ",age_coef," b: ",obesity_coef," c: ",smoking_coef," intercept: ",avg_intercept)

In [0]:
def calculate_insurance(age,obesity,smoking):
    y=(age_coef*age)+(obesity_coef*obesity)+(smoking_coef*smoking)+avg_intercept
    return y

In [0]:
a=int(input("enter the age",end='\n'))
if(a>64):
   print("your entered Age is too high,it might have a chance to giving wrong prediction",end='\n')
   print("Avoid to giving the policy to the age above 65",end='\n')
b=float(input("enter the bmi value limit is(15 to 30)",end='\n'))
c=int(input("Smoker 1 or Not 0"))
if((b>25)or(b<15)):
   print("your entered BMI Value is Different so,it might have a chance to giving wrong prediction",end='\n')
print(calculate_insurance(a,b,c))